In [1]:
!pip install shap

     |████████████████████████████████| 356 kB 10.8 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491631 sha256=1392fdc9342035be477f1c4fb19d0d212aea76733106df909749bda25e6cec25
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from google.colab import files
import pandas as pd
import json
import re
import ast
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
import shap
from sklearn import metrics

In [3]:
def process_spectra(file_obj, file_name):
    file_string = str(file_obj[file_name])

    return file_string.split("\\n")

In [4]:
def process_matrix(file_obj, file_name):
    file_string = str(file_obj[file_name])

    result = file_string.split("\\n")
    result[0] = result[0].replace("b'", "")

    for i in range(len(result)):
        result[i] = result[i].split(" ")
        result[i] = [int(x) if x.isdigit() else x for x in result[i]]
        lastel = result[i][-1]
        if lastel == "+":
            result[i][-1] = "PASS"
        elif lastel == "-":
            result[i][-1] = "FAIL"
    
    return result

In [5]:
def make_line_in_txt_shap(shap_value_exp_obj_instance, instance_name):
    feature_indexes = []
    new_list = []
    for i in range(len(df.iloc[instance_name])):
        if df.iloc[instance_name][i] == 1:
            feature_indexes.append(i)
    for i in feature_indexes:
        new_list.append(";".join([list(x.columns)[i], str(shap_value_exp_obj_instance[i].values)]))
    return new_list    

In [6]:
def write_output_file(filename, obj, is_reverse=False):
    with open(filename, 'w') as file:
        sorted_dict = sorted(obj.items(), key=lambda x: x[1], reverse=is_reverse)
        for i in sorted_dict:
            file.write(";".join([i[0], str(i[1])]) + "\n")

In [7]:
spectra_file = files.upload()
matrix_file = files.upload()

Saving spectra to spectra


Saving matrix to matrix


In [8]:
spectra_list = process_spectra(spectra_file, list(spectra_file.keys())[0])
spectra_list[-1] = "Pass/Fail"
spectra_list[0] = spectra_list[0].replace("b'", "")

In [9]:
matrix_list = process_matrix(matrix_file, list(matrix_file.keys())[0])

In [10]:
matrix_list.pop()

["'"]

In [11]:
df = pd.DataFrame(data=matrix_list, columns=spectra_list)

In [12]:
df

,org.apache.commons.math3.util.MathArrays$1#674,org.apache.commons.math3.util.MathArrays$1#678,org.apache.commons.math3.util.MathArrays$1#680,org.apache.commons.math3.util.MathArrays$1#681,org.apache.commons.math3.util.MathArrays$1#683,org.apache.commons.math3.util.MathArrays$1#684,org.apache.commons.math3.util.MathArrays$1#687,org.apache.commons.math3.util.MathArrays$1#689,org.apache.commons.math3.util.MathArrays#51,org.apache.commons.math3.util.MathArrays#88,org.apache.commons.math3.util.MathArrays#89,org.apache.commons.math3.util.MathArrays#92,org.apache.commons.math3.util.MathArrays#93,org.apache.commons.math3.util.MathArrays#94,org.apache.commons.math3.util.MathArrays#96,org.apache.commons.math3.util.MathArrays#110,org.apache.commons.math3.util.MathArrays#111,org.apache.commons.math3.util.MathArrays#114,org.apache.commons.math3.util.MathArrays#115,org.apache.commons.math3.util.MathArrays#116,org.apache.commons.math3.util.MathArrays#118,org.apache.commons.math3.util.MathArrays#132,org.apache.commons.math3.util.MathArrays#133,org.apache.commons.math3.util.MathArrays#136,org.apache.commons.math3.util.MathArrays#137,org.apache.commons.math3.util.MathArrays#138,org.apache.commons.math3.util.MathArrays#140,org.apache.commons.math3.util.MathArrays#154,org.apache.commons.math3.util.MathArrays#155,org.apache.commons.math3.util.MathArrays#158,org.apache.commons.math3.util.MathArrays#159,org.apache.commons.math3.util.MathArrays#160,org.apache.commons.math3.util.MathArrays#162,org.apache.commons.math3.util.MathArrays#173,org.apache.commons.math3.util.MathArrays#174,org.apache.commons.math3.util.MathArrays#175,org.apache.commons.math3.util.MathArrays#177,org.apache.commons.math3.util.MathArrays#188,org.apache.commons.math3.util.MathArrays#189,org.apache.commons.math3.util.MathArrays#190,...,org.apache.commons.math3.random.RandomDataImpl#127,org.apache.commons.math3.random.RandomDataImpl#128,org.apache.commons.math3.random.RandomDataImpl#136,org.apache.commons.math3.random.RandomDataImpl#156,org.apache.commons.math3.random.RandomDataImpl#161,org.apache.commons.math3.random.RandomDataImpl#166,org.apache.commons.math3.random.RandomDataImpl#186,org.apache.commons.math3.random.RandomDataImpl#191,org.apache.commons.math3.random.RandomDataImpl#196,org.apache.commons.math3.random.RandomDataImpl#213,org.apache.commons.math3.random.RandomDataImpl#218,org.apache.commons.math3.random.RandomDataImpl#233,org.apache.commons.math3.random.RandomDataImpl#248,org.apache.commons.math3.random.RandomDataImpl#265,org.apache.commons.math3.random.RandomDataImpl#279,org.apache.commons.math3.random.RandomDataImpl#293,org.apache.commons.math3.random.RandomDataImpl#307,org.apache.commons.math3.random.RandomDataImpl#320,org.apache.commons.math3.random.RandomDataImpl#336,org.apache.commons.math3.random.RandomDataImpl#363,org.apache.commons.math3.random.RandomDataImpl#383,org.apache.commons.math3.random.RandomDataImpl#401,org.apache.commons.math3.random.RandomDataImpl#415,org.apache.commons.math3.random.RandomDataImpl#431,org.apache.commons.math3.random.RandomDataImpl#447,org.apache.commons.math3.random.RandomDataImpl#461,org.apache.commons.math3.random.RandomDataImpl#462,org.apache.commons.math3.random.RandomDataImpl#472,org.apache.commons.math3.random.RandomDataImpl#473,org.apache.commons.math3.random.RandomDataImpl#485,org.apache.commons.math3.random.RandomDataImpl#486,org.apache.commons.math3.random.RandomDataImpl#493,org.apache.commons.math3.random.RandomDataImpl#494,org.apache.commons.math3.random.RandomDataImpl#518,org.apache.commons.math3.random.RandomDataImpl#519,org.apache.commons.math3.random.RandomDataImpl#532,org.apache.commons.math3.random.RandomDataImpl#550,org.apache.commons.math3.random.RandomDataImpl#565,org.apache.commons.math3.random.RandomDataImpl#581,Pass/Fail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PASS
1,0,0,0,0,0,0,0,0,

In [13]:
x = df[spectra_list[:len(spectra_list)-1]]
y = df["Pass/Fail"]

In [14]:
fails = []
passes = []
i = 0
for result in y:
    if result == "FAIL":
        fails.append(i)
    else:
        passes.append(i)
    i += 1 

In [15]:
if len(fails) == 1:
    df = df.append(df.iloc[fails[0]])
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [16]:
if (df.shape[0] < 6):
    df = df.append(df)
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

In [19]:
model = KNeighborsClassifier()

In [20]:
model.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [21]:
y_pred=model.predict(x_test)

In [22]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        PASS       1.00      1.00      1.00         5

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [23]:
shap_to_txt = {}

explainer = shap.KernelExplainer(model.predict_proba, x_train)
shap_values = explainer.shap_values(x)


for i in fails:
    temp_list = []
    for index, j in enumerate(shap_values[0][i]):
        temp_list.append(spectra_list[index] + ";" + str(j))
    shap_to_txt[x.iloc[i].name] = temp_list

In [30]:
shap_to_txt

{18: ['org.apache.commons.math3.util.MathArrays$1#674;0.0',
  'org.apache.commons.math3.util.MathArrays$1#678;0.0',
  'org.apache.commons.math3.util.MathArrays$1#680;0.0',
  'org.apache.commons.math3.util.MathArrays$1#681;0.0',
  'org.apache.commons.math3.util.MathArrays$1#683;0.0',
  'org.apache.commons.math3.util.MathArrays$1#684;0.0',
  'org.apache.commons.math3.util.MathArrays$1#687;0.0',
  'org.apache.commons.math3.util.MathArrays$1#689;0.0',
  'org.apache.commons.math3.util.MathArrays#51;0.0',
  'org.apache.commons.math3.util.MathArrays#88;0.0',
  'org.apache.commons.math3.util.MathArrays#89;0.0',
  'org.apache.commons.math3.util.MathArrays#92;0.0',
  'org.apache.commons.math3.util.MathArrays#93;0.0',
  'org.apache.commons.math3.util.MathArrays#94;0.0',
  'org.apache.commons.math3.util.MathArrays#96;0.0',
  'org.apache.commons.math3.util.MathArrays#110;0.0',
  'org.apache.commons.math3.util.MathArrays#111;0.0',
  'org.apache.commons.math3.util.MathArrays#114;0.0',
  'org.apache.c

In [31]:
shap_lines_output_val = {}

for i in shap_to_txt:
    for index, value in enumerate(shap_to_txt[i]):
        feature_shapval = value.split(";")
        feature = feature_shapval[0]
        shapval = feature_shapval[1]
        if feature not in shap_lines_output_val:
            shap_lines_output_val[feature] = []
        shap_lines_output_val[feature].append(float(shapval))

In [32]:
shap_means = {}
shap_max = {}
shap_min = {}

for i in shap_lines_output_val:
    if i not in shap_means:
        shap_means[i] = []
    if i not in shap_min:
        shap_min[i] = shap_lines_output_val[i][0]
    if i not in shap_max:
        shap_max[i] = shap_lines_output_val[i][0]
    for j in shap_lines_output_val[i]:
        shap_means[i].append(j)
        if abs(shap_max[i]) < abs(j):
            shap_max[i] = j
        if abs(shap_min[i]) > abs(j):
            shap_min[i] = j

for i in shap_means:
    shap_means[i] = sum(shap_means[i]) / len(shap_means[i])

In [35]:
with open("Math-8_knn-shap_results.txt", "w") as file:
    for i in shap_to_txt:
        file.write(str(i) + "\n")
        for j in range(len(shap_to_txt[i])):
            file.write(shap_to_txt[i][j] + "\n")

In [36]:
write_output_file('Math-8_knn-shap_mean.txt', shap_means, is_reverse=True)
write_output_file('Math-8_knn-shap_min.txt', shap_min, is_reverse=True)
write_output_file('Math-8_knn-shap_max.txt', shap_max, is_reverse=True)